In [5]:
# Import findspark and initialize.
import findspark
findspark.init()

In [7]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/27 22:10:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Add the file to be downloaded with Spark
spark.sparkContext.addFile(url)

# Use the SparkFiles class to get the path where the file is downloaded
file_path = "file://" + SparkFiles.get("home_sales_revised.csv")

# Read the CSV file into a PySpark DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Show the first few rows of the DataFrame to confirm
df.show(5)


25/01/27 22:19:51 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [13]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [15]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

query = """
    SELECT 
        YEAR(date) AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    WHERE 
        bedrooms = 4
    GROUP BY 
        YEAR(date)
    ORDER BY 
        year_built
"""

# Execute the SQL query
result = spark.sql(query)

# Show the result
result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2019| 300263.7|
|      2020|298353.78|
|      2021|301819.44|
|      2022|296363.88|
+----------+---------+



In [17]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
# Create the SQL query to calculate the average price of 3-bedroom, 3-bathroom homes per year built
query = """
    SELECT 
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    WHERE 
        bedrooms = 3 AND bathrooms = 3
    GROUP BY 
        date_built
    ORDER BY 
        year_built
"""

# Execute the SQL query
result = spark.sql(query)

# Show the result
result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [19]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

query = """
    SELECT 
        date_built AS year_built,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    WHERE 
        bedrooms = 3 
        AND bathrooms = 3 
        AND floors = 2 
        AND sqft_living >= 2000
    GROUP BY 
        date_built
    ORDER BY 
        year_built
"""

# Execute the SQL query
result = spark.sql(query)

# Show the result
result.show()


+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [21]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """
    SELECT 
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    GROUP BY 
        view
    HAVING 
        AVG(price) >= 350000
    ORDER BY 
        view DESC
"""

# Execute the SQL query
result = spark.sql(query)

# Show the result
result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.340054988861084 seconds ---


In [23]:
# 7. Cache the the temporary table home_sales.

df.cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [25]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [27]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

query_cached = """
    SELECT 
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales
    GROUP BY 
        view
    HAVING 
        AVG(price) >= 350000
    ORDER BY 
        view DESC
"""

# Execute the cached query
result_cached = spark.sql(query_cached)

# Show the cached result
result_cached.show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.7436840534210205 seconds ---


In [31]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

# Define the path where you want to save the Parquet file
output_path = "../Desktop/Data_Science/Mod_22/partitioned_home_sales_data"

# Partition the DataFrame by "date_built" and save it as a Parquet file
df.write.partitionBy("date_built").parquet(output_path)

In [33]:
# 11. Read the formatted parquet data.

# Define the path where the Parquet data is stored
input_path = "../Desktop/Data_Science/Mod_22/partitioned_home_sales_data"

# Read the Parquet data into a DataFrame
df_parquet = spark.read.parquet(input_path)

# Show the first few rows of the DataFrame to confirm it was read correctly
df_parquet.show(5)


+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
+--------------------+----------+------+--------+---------+-----------+--------+

In [35]:
# 12. Create a temporary table for the parquet data.

df_parquet.createOrReplaceTempView("home_sales_parquet")

In [39]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
query_parquet = """
    SELECT 
        view,
        ROUND(AVG(price), 2) AS avg_price
    FROM 
        home_sales_parquet
    GROUP BY 
        view
    HAVING 
        AVG(price) >= 350000
    ORDER BY 
        view DESC
"""

# Execute the SQL query on the Parquet data
result_parquet = spark.sql(query_parquet)

# Show the result
result_parquet.show()

# Measure the runtime of the query on the Parquet data




print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.8811640739440918 seconds ---


In [41]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [43]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")

# Print the result
if is_cached:
    print("The 'home_sales' table is still cached.")
else:
    print("The 'home_sales' table is no longer cached.")

The 'home_sales' table is no longer cached.
